<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/2.1_Modeling_and_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling and Predictions
[Notebook 1: EDA and Data Cleaning](./1_EDA_and_DataCleaning.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling_and_Predictions.ipynb)

Notebook 3: Discussion and Technical Report

... Was geschieht in diesem Dokument? ...

1. Features
2. Resampling
3. Modeling
4. Scaling
5. Lagged features
6. Train test split, fit models, evaluate

## Verbinden mit der Google-Drive und Import der benötigten Module:

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## import modules
import os
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import keras
from keras.utils import to_categorical, normalize
from keras.layers import Input, Dense, LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split

In [3]:
## Pfad an dem die Daten liegen
drive_path = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'

name_Messwerte = 'UVI_Messdaten.csv'
name_Vorhersage = 'Vorhersage.csv'
name_Solar_Messwerte = 'Solys_Messdaten.csv'

In [4]:
# check tensorflow version
print("tensorflow version:", tf.__version__)
# check available gpu
gpus =  tf.config.list_physical_devices('GPU')
print("available gpus:", gpus)
# limit the gpu usage, prevent it from allocating all gpu memory for a simple model
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# check number of cpus available
print("available cpus:", os.cpu_count())

tensorflow version: 2.15.0
available gpus: []
available cpus: 2


In [5]:
# change model name for different models

# model_name = 'SUNSET_forecast_2017_2019_data'
# output_folder = os.path.join(pardir,"model_output", model_name)
# if os.path.isdir(output_folder)==False:
#     os.makedirs(output_folder)

## Import der Daten:

In [6]:
# Import der Daten für die Modellbildung
df_UVI = pd.read_csv(drive_path + name_Messwerte)
df_Solys = pd.read_csv(drive_path + name_Solar_Messwerte)
df_CAMS = pd.read_csv(drive_path + name_Vorhersage)

# Messzeitpunkt in Datetime umwandeln
df_UVI.set_index('Datetime', inplace=True)
df_Solys.set_index('Datetime', inplace=True)
df_CAMS.set_index('Datetime', inplace=True)

# Index in Datetime umwandeln
df_UVI.index = pd.to_datetime(df_UVI.index)
df_Solys.index = pd.to_datetime(df_Solys.index)
df_CAMS.index = pd.to_datetime(df_CAMS.index)

## Allgemeine Informationen:

In [7]:
# Allgemeine Informationen über die Messdaten
round(df_UVI.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,120028.0,60013.50,34649.24,0.00,30006.75,60013.50,90020.25,120027.00
Messzeitpunkt,120028.0,40465.35,13493.43,12360.00,29460.00,40500.00,51480.00,68220.00
erythem,120026.0,0.03,0.04,0.00,0.00,0.02,0.05,0.21
UVI,120026.0,1.33,1.65,0.00,0.15,0.61,1.90,8.54
DiffGreater2,120028.0,0.01,0.10,0.00,0.00,0.00,0.00,1.00
SZA,120028.0,64.29,17.05,27.36,51.10,67.07,77.85,97.87
time_sin,120028.0,0.12,0.70,-1.00,-0.57,0.20,0.83,1.00
time_cos,120028.0,-0.58,0.40,-1.00,-0.92,-0.70,-0.32,0.62
date_sin,120028.0,-0.07,0.70,-1.00,-0.70,-0.25,0.70,1.00
date_cos,120028.0,-0.18,0.69,-1.00,-0.81,-0.33,0.48,1.00


In [8]:
# Allgemeine Informationen über die Messdaten
round(df_Solys.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
Glo,97260.0,254.13,250.23,0.0,52.90,167.03,387.52,1232.83
Dif,97260.0,125.54,109.78,0.0,45.45,91.53,178.38,655.32
Glo_SPLite,97260.0,259.03,248.57,0.0,57.96,173.37,397.06,1189.92
Dir,97260.0,249.70,324.94,0.0,2.04,17.96,531.48,1016.76
Temp,97260.0,16.69,8.59,-7.8,10.60,16.76,23.00,54.32


In [9]:
# Allgemeine Informationen über die Vorhersagedaten
round(df_CAMS.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
aod469,4307.0,0.20,0.15,0.02,0.10,0.15,0.26,1.31
aod550,4307.0,0.16,0.12,0.01,0.08,0.13,0.21,1.08
aod670,4307.0,0.12,0.09,0.01,0.06,0.10,0.16,0.82
aod865,4307.0,0.01,0.00,0.00,0.01,0.01,0.01,0.01
uvbed,4307.0,1.26,1.56,-0.00,0.12,0.60,1.83,7.56
uvbedcs,4307.0,1.73,1.91,-0.00,0.21,0.90,2.79,7.67
hcc,4307.0,0.43,0.42,0.00,0.00,0.29,0.94,1.00
lcc,4307.0,0.41,0.40,0.00,0.01,0.28,0.87,1.00
mcc,4307.0,0.32,0.36,0.00,0.00,0.16,0.61,1.00
tcc,4307.0,0.72,0.35,0.00,0.46,0.93,1.00,1.00


In [10]:
# Daten zusammen führen
df_model = pd.concat([df_UVI[['UVI', 'SZA', 'time_sin', 'time_cos', 'date_sin', 'date_cos']], df_Solys[['Glo', 'Glo_SPLite']]], axis = 1)

In [11]:
df_model.dtypes     # solange alle Splaten das slebe Gleikommeformat haben kann der Dataframe wie ein np.array angesehen werden.

UVI           float64
SZA           float64
time_sin      float64
time_cos      float64
date_sin      float64
date_cos      float64
Glo           float64
Glo_SPLite    float64
dtype: object

In [12]:
# Extrahieren der Spalten, die für das Modell genutzt werden sollen
cols = list(df_model)[1:]

In [18]:
cols

['SZA', 'time_sin', 'time_cos', 'date_sin', 'date_cos', 'Glo', 'Glo_SPLite']

## Normalisieren der Daten:

In [13]:
# Normalisieren der Daten (Was gibt es noch für Scaler)
normalize(df_model, order = 1)

,UVI,SZA,time_sin,time_cos,date_sin,date_cos,Glo,Glo_SPLite
Datetime,,,,,,,,
2022-06-15 07:21:00,0.002038,0.046560,0.000794,-0.000293,0.000238,-0.000812,0.469116,0.480150
2022-06-15 07:23:00,0.002071,0.046030,0.000787,-0.000298,0.000237,-0.000807,0.469231,0.480539
2022-06-15 07:25:00,0.002073,0.045492,0.000779,-0.000303,0.000235,-0.000802,0.469698,0.480617
2022-06-15 07:27:00,0.002116,0.045006,0.000773,-0.000308,0.000234,-0.000798,0.469936,0.480829
2022-06-15 07:29:00,0.002119,0.044234,0.000761,-0.000311,0.000231,-0.000789,0.470385,0.481168
...,...,...,...,...,...,...,...,...
2023-08-19 05:48:00,0.000543,0.138086,0.001767,0.000093,-0.001309,-0.001190,0.411438,0.445574
2023-08-19 05:50:00,0.000551,0.137119,0.001763,0.000077,-0.001305,-0.001187,0.411886,0.446112
2023-08-19 05:52:00,0.000595,0.146476,0.001891,0.000066,-0.001400,-0.001273,0.407513,0.440786


## Aufteilen der Daten:

In [25]:
trainX = []
trainY = []

n_future = 1
n_past = 15

In [36]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
   # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
  if i == 0:
    names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
  else:
    names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

In [37]:
series_to_supervised(df_train, 1, 1)

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t)
Datetime,,,,,,,,,,,,,,
2022-07-30 11:41:00,33.459624,0.317305,-0.948324,0.752859,-0.658182,431.891667,456.200000,32.364694,0.082808,-0.996566,-0.470208,-0.882556,849.450000,832.250000
2022-06-24 17:55:00,72.208127,0.996195,-0.087156,-0.691931,-0.721964,169.708333,174.783333,75.289285,-0.999762,-0.021815,0.128705,-0.991683,123.600000,132.075000
2023-04-04 16:42:00,75.289285,-0.999762,-0.021815,0.128705,-0.991683,123.600000,132.075000,77.297547,-0.942641,-0.333807,0.998840,-0.048151,179.650000,189.141667
2023-04-03 06:30:00,77.297547,-0.942641,-0.333807,0.998840,-0.048151,179.650000,189.141667,77.218362,0.991445,-0.130526,0.999556,-0.029799,194.175000,197.825000
2022-10-09 10:15:00,77.218362,0.991445,-0.130526,0.999556,-0.029799,194.175000,197.825000,58.756185,0.442289,-0.896873,-0.990221,0.139507,531.325000,537.733333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17 13:03:00,81.677692,0.887011,-0.461749,-0.760067,0.649844,70.633333,75.025000,65.020375,-0.271440,-0.962455,0.736054,0.676923,298.983333,299.908333
2023-07-21 10:22:00,65.020375,-0.271440,-0.962455,0.736054,0.676923,298.983333,299.908333,33.778088,0.414693,-0.909961,-0.328345,-0.944558,428.475000,434.700000
2023-08-18 15:18:00,33.778088,0.414693,-0.909961,-0.328345,-0.944558,428.475000,434.700000,58.560967,-0.760406,-0.649448,-0.728953,-0.684564,475.075000,482.466667


In [28]:
# Daten in Input und Outpu Vektor aufteilen
for i in range(n_past, len(df_train) - n_future + 1):
    trainX.append(df_train.iloc[i - n_past:i, 0:df_train.shape[1]])
    trainY.append(df_train.iloc[i + n_future - 1:i + n_future, 0])

In [32]:
def create_dataset(dataset, look_back=15):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [33]:
X, Y = create_dataset(df_train)

InvalidIndexError: ignored

In [16]:
# target = tf.convert_to_tensor(target)
# numeric_features = tf.convert_to_tensor(df_train)

In [17]:
x_train = train_data[[i for i in range(7)]].values

NameError: ignored

In [ ]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features, target))

for row in numeric_dataset.take(3):
  print(row)

In [ ]:
BATCH_SIZE = 15
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

In [ ]:
numeric_batches

## Building a Network:

Datenformatierung:
**Input**: shape: 15 timesteps und 8 features

In [ ]:
# define the model architecture using tf.keras API
def simple_model():
    # Definition der Inputlayer für das Netzwerk
    x_in = Input(shape = (15,8), name = "input_layer", )

    return model

In [ ]:
# show model architecture
simple_model().summary()

## Modell fitten:

In [ ]:
# define model characteristics
num_filters = 24
kernel_size = [3,3]
pool_size = [2,2]
strides = 2
dense_size = 1024
drop_rate = 0.4

# define training time parameters
num_epochs = 200 #(The maximum epoches set to 200 and there might be early stopping depends on validation loss)
num_fold = 10 # 10-fold cross-validation
batch_size = 256
learning_rate = 3e-06

## Validierung des Netzwerks: